
# Introduction to Artificial Intelligence - Homework Assignment 01 (20pts.)
- NETID:
- Name:

This assignment covers the following topics:
- A* Search
- Constraint Satisfaction Problems

Please complete all sections. Some questions will require written answers, while others will involve coding. Be sure to run your code cells to verify your solutions.


"Quack Quack Quack"

You toss another piece of bread to the ducks, beginning to gather around you, in Central Park.
Suddenly, you feel a tap on your shoulder. You turn around, and there is a man standing there.

"Detective Student?" he asks, "We have a slight problem, there's been a murder."

You quickly slip your flask into your jacket.

"No time for questions, we need you to get to the William Theisen-Floyd Estate as soon as possible, cost is no object."

You thankfully sigh, knowing that those new congestion charges have been draining your bank account. As you start walking away and try to figure out how to get out to the estate (unfortunately the office said a blade was out of the question), you recall an algorithm that may be able to help you get to the estate! 

## Section 1: Search Algorithms (8 pts.)

In this section, you'll need to use the provided map to get from Central Park to the William Theisen-Floyd Estate in the Hamptons in the least amount of time possible.

<img src="https://raw.githubusercontent.com/nd-cse-30124-sp25/nd-cse-30124-sp25.github.io/refs/heads/main/static/svg/hw01_map_graph.svg" alt="Map" style="width: 100%; max-width: 800px; height: auto;">


### Part 1: Uninformed Search (Djikstra's Algorithm) (5 pts.)

In the cell below, write an implementation of Djikstra's algorithm and calculate:
* The number of nodes visited
* The number of edges evaluated
* The best path from Central Park to the Estate
* The total travel time for the best path

In [28]:
import heapq

# Example usage:
# Graph represented as an adjacency list
# Each key is a node, and the value is a dictionary of neighbors and their edge weights
node_number_to_name = {
    0: 'Central Park',
    1: 'Grand Central Station',
    2: 'LGA',
    3: 'Norwalk',
    4: 'Jamaica',
    5: 'JFK',
    6: 'Bridgeport',
    7: 'BDR',
    8: 'Port Jefferson',
    9: 'Huntington',
    10: 'Massapequa',
    11: 'KISP',
    12: 'Patchogue',
    13: 'William Theisen-Floyd Estate',
}

#Adjacency List for the graph
# Node: {Neighbor N: (Taxi, Train, Ferry, Flight)}

graph = {
    0: {1: (8, 22, 0, 0)},  # Central Park to Grand Central (Taxi: 10 min, Train: 5 min)
    1: {2: (16, 48, 0, 0), 3: (63, 64, 0, 0), 4: (34, 43, 0, 0), 9: (61, 100, 0, 0)},  # Grand Central connections
    2: {5: (0, 0, 0, 31), 7: (0, 0, 0, 36), 11: (0, 0, 0, 35)},  # LGA connections
    3: {6: (20, 35, 0, 0), 9: (0, 0, 26, 0)},  # Norwalk connections
    4: {5: (15, 23, 0, 0), 9: (47, 72, 0, 0), 10: (41, 45, 0, 0)},  # Jamaica connections
    5: {7: (0, 0, 0, 36), 11: (0, 0, 0, 34), 13: (0, 0, 90, 0)},  # JFK connections
    6: {8: (0, 0, 75, 0)},  # Bridgeport connections
    7: {6: (10, 48, 0, 0), 11: (0, 0, 0, 33)},  # BDR connections
    8: {12: (28, 56, 0, 0)},  # Port Jefferson connections
    9: {8: (48, 60, 0, 0)},  # Huntington connections
    10: {12: (34, 52, 0, 0), 13: (0, 0, 40, 0)},  # Massapequa connections
    11: {12: (13, 24, 0, 0)},  # KISP connections
    12: {13: (23, 47, 0, 0)},  # Patchogue connections
    13: {}  # William Theisen-Floyd Estate connections
}

def dijkstra_multi_mode(graph, start):
    """
    Dijkstra's algorithm with support for multiple transport modes.

    Args:
    - graph: Dictionary representing the graph with multi-mode costs.
    - start: Starting node.

    Returns:
    - distances: Dictionary with the shortest distance to each node from the start.
    - paths: Dictionary with the best path to each node.
    - nodes_visited: Count of how many nodes were visited.
    - edges_evaluated: Count of how many edges were evaluated.
    """
    # Priority queue to store (distance, vertex) pairs
    priority_queue = []
    heapq.heappush(priority_queue, (0, start))

    # Distances dictionary, initialized with infinity for all nodes except the start
    distances = {vertex: float('infinity') for vertex in graph}
    distances[start] = 0

    # Paths dictionary to track the best route to each node
    paths = {vertex: [] for vertex in graph}
    paths[start] = [start]

    # Track visited nodes and counters
    visited = set()
    nodes_visited = 0  # Counter for visited nodes
    edges_evaluated = 0  # Counter for evaluated edges

    while priority_queue:
        # Get the vertex with the smallest distance
        current_distance, current_vertex = heapq.heappop(priority_queue)

        if current_vertex in visited:
            continue

        visited.add(current_vertex)
        nodes_visited += 1  # Increment the visited nodes counter

        # Check all neighbors of the current vertex
        for neighbor, costs in graph[current_vertex].items():
            edges_evaluated += 1  # Increment the evaluated edges counter

            # Find the minimum cost across all modes of transport
            min_cost = min(cost for cost in costs if cost > 0)  # Ignore unavailable connections (cost=0)

            if min_cost > 0:  # Only consider valid connections
                distance = current_distance + min_cost
                # Update the distance if it's smaller than the previously recorded distance
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    paths[neighbor] = paths[current_vertex] + [neighbor]
                    heapq.heappush(priority_queue, (distance, neighbor))

    return distances, paths, nodes_visited, edges_evaluated

def print_best_path_to_estate(paths, graph, node_number_to_name, start, destination):
    """
    Prints the best path from the start node to the destination node with transport modes and times,
    without repeating node names.

    Args:
    - paths: Dictionary of shortest paths to each node.
    - graph: The adjacency list with costs for transport modes.
    - node_number_to_name: Dictionary mapping node numbers to their names.
    - start: The start node number.
    - destination: The destination node number.
    """
    mode_names = ["TAXI", "TRAIN", "FERRY", "FLIGHT"]

    # Get the best path from start to destination
    path = paths.get(destination, None)
    if not path or path[0] != start:
        print("No valid path found.")
        return

    path_segments = []
    path_segments.append(node_number_to_name[start])
    total_time = 0  # Track the total travel time

    for i in range(len(path) - 1):
        current = path[i]
        next_node = path[i + 1]
        costs = graph[current][next_node]

        # Find the mode of transport used (the minimum non-zero cost)
        mode_index = costs.index(min(cost for cost in costs if cost > 0))
        mode = mode_names[mode_index]
        time = costs[mode_index]  # Time for the chosen mode

        # Append the segment with transport mode and time
        path_segments.append(f" ---[{mode}, {time} min]--> {node_number_to_name[next_node]}")
        total_time += time

    # Join the path segments and print the result
    print(f"Best path from {node_number_to_name[start]} to {node_number_to_name[destination]}:")
    print("".join(path_segments))
    print(f"Total travel time: {total_time} minutes")

# Using the previously defined `graph`
start_node = 0  # Central Park
distances, paths, nodes_visited, edges_evaluated = dijkstra_multi_mode(graph, start_node)

print(f"Number of nodes visited: {nodes_visited}")
print(f"Number of edges evaluated: {edges_evaluated}")

# Call the function to print the best path from Central Park to the Estate
print_best_path_to_estate(paths, graph, node_number_to_name, start=0, destination=13)

Number of nodes visited: 14
Number of edges evaluated: 25
Best path from Central Park to William Theisen-Floyd Estate:
Central Park ---[TAXI, 8 min]--> Grand Central Station ---[TAXI, 16 min]--> LGA ---[FLIGHT, 35 min]--> KISP ---[TAXI, 13 min]--> Patchogue ---[TAXI, 23 min]--> William Theisen-Floyd Estate
Total travel time: 95 minutes


### Part 2: Informed Search (A*) (1 pts.)

Copy your code from above and then modify it to make use of the heuristic information provided in the table below. Calculate:

* The number of nodes visited
* The number of edges evaluated
* The best path from Central Park to the Estate
* The total travel time for the best path

In [27]:
heuristics = {
    0: 90,   # Central Park
    1: 80,   # Grand Central Station
    2: 70,   # LGA
    3: 75,   # Norwalk
    4: 65,   # Jamaica
    5: 50,   # JFK
    6: 60,   # Bridgeport
    7: 55,   # BDR
    8: 45,   # Port Jefferson
    9: 60,   # Huntington
    10: 30,  # Massapequa
    11: 40,  # KISP
    12: 20,  # Patchogue
    13: 0    # William Theisen-Floyd Estate
}

import heapq

def a_star(graph, heuristics, start, destination):
    """
    A* algorithm for finding the shortest path in a graph.
    
    Args:
    - graph: Dictionary representing the adjacency list with travel costs for multiple modes.
    - heuristics: Dictionary of heuristic values for each node.
    - start: The starting node.
    - destination: The destination node.
    
    Returns:
    - path: List of nodes representing the shortest path from start to destination.
    - cost: Total travel cost of the shortest path.
    - nodes_visited: Number of nodes visited.
    - edges_evaluated: Number of edges evaluated.
    """
    # Priority queue for nodes to explore (f(n), node, g(n), path)
    priority_queue = []
    heapq.heappush(priority_queue, (heuristics[start], start, 0, [start]))

    # Dictionary to store the best g(n) value for each node
    g_costs = {node: float('inf') for node in graph}
    g_costs[start] = 0

    # Track visited nodes and count nodes visited and edges evaluated
    visited = set()
    nodes_visited = 0
    edges_evaluated = 0

    while priority_queue:
        # Get the node with the smallest f(n)
        _, current_node, current_g_cost, path = heapq.heappop(priority_queue)

        # If the current node has already been visited, skip it
        if current_node in visited:
            continue

        # Mark the current node as visited
        visited.add(current_node)
        nodes_visited += 1  # Increment the visited nodes counter

        # If we reach the destination, return the results
        if current_node == destination:
            return path, current_g_cost, nodes_visited, edges_evaluated

        # Explore neighbors
        for neighbor, costs in graph[current_node].items():
            edges_evaluated += 1  # Increment the edges evaluated counter

            # Find the minimum cost among all available transport modes
            transport_cost = min(cost for cost in costs if cost > 0)

            if transport_cost > 0:
                new_g_cost = current_g_cost + transport_cost  # Update g(n)
                
                # Only proceed if this path improves the g_cost
                if new_g_cost < g_costs[neighbor]:
                    g_costs[neighbor] = new_g_cost
                    f_cost = new_g_cost + heuristics[neighbor]  # Compute f(n)
                    heapq.heappush(priority_queue, (f_cost, neighbor, new_g_cost, path + [neighbor]))

    # If no path is found
    return None, float('inf'), nodes_visited, edges_evaluated

# Find the best path from Central Park (0) to William Theisen-Floyd Estate (13)
path, cost, nodes_visited, edges_evaluated = a_star(graph, heuristics, start=0, destination=13)

print(f"Number of nodes visited: {nodes_visited}")
print(f"Number of edges evaluated: {edges_evaluated}")

print_best_path_to_estate({13: path}, graph, node_number_to_name, start=0, destination=13)


Number of nodes visited: 6
Number of edges evaluated: 10
Best path from Central Park to William Theisen-Floyd Estate:
Central Park ---[TAXI, 8 min]--> Grand Central Station ---[TAXI, 16 min]--> LGA ---[FLIGHT, 35 min]--> KISP ---[TAXI, 13 min]--> Patchogue ---[TAXI, 23 min]--> William Theisen-Floyd Estate
Total travel time: 95 minutes


### Part 3: Were the results of the two algorithms the same or different? Why do you think this is? (2 pt.)

[ANSWER]


## Problem 2: Constraint Satisfaction Problems (12 pts.)

On your way over to the estate, you read over the police report:

The suspects are three men (Dr. Bui, Dr. Dingler, Dr. Morrison) and three women (Dr. Chambers, Dr. Kumar, Dr. Levis). Each person was in a different room (Bathroom, Dining Room, Kitchen, Living Room, Pantry, Study). A suspected weapon was found in each room (Bag, Firearm, Gas, Knife, Poison, Rope). 

Armed with this knowledge, you arrive at the estate and begin methodically exploring the house. As you explore, you slowly piece together a series of clues:

1. The man in the kitchen was not found with the rope, knife, or bag. Which weapon, then, which was not the firearm, was found in the kitchen?

2. Barbara was either in the study or the bathroom; Yolanda was in the other. Which room was Barbara found in?

3. The person with the bag, who was not Barbara nor George, was not in the bathroom nor the dining room. Who had the bag in the room with them?

4. The woman with the rope was found in the study. Who had the rope?

5. The weapon in the living room was found with either John or George. What weapon was in the living room?

6. The knife was not in the dining room. So where was the knife?

7. Yolanda was not with the weapon found in the study nor the pantry. What weapon was found with Yolanda?

8. The firearm was in the room with George. In which room was the firearm found?

9. It was discovered that Mr. Boddy was gassed in the pantry. The suspect found in that room was the murderer.

As you ponder the clues (and the scenery, drink in hand) inspiration strikes! You recall learning another algorithm in class that could be used.

In the code cell below implement the backtracking algorithm to solve the murder mystery!

In [30]:
from collections import deque

# Define variables
people = ["George", "John", "Robert", "Barbara", "Christine", "Yolanda"]
rooms = ["Bathroom", "Dining Room", "Kitchen", "Living Room", "Pantry", "Study"]
weapons = ["Bag", "Firearm", "Gas", "Knife", "Poison", "Rope"]

# Set of all assignments
assignments = []
num_checks = 0

# Define domains
domains = {
    person: set(rooms) for person in people
}
domains.update({
    room: set(weapons) for room in rooms
})

# Clues as constraints
def is_valid(assignment):
    """
    Check if the current assignment satisfies all clues.
    """
    global num_checks
    num_checks += 1

    # Extract assignments for people, rooms, and weapons
    person_to_room = {person: room for person, room, weapon in assignment}
    person_to_weapon = {person: weapon for person, room, weapon in assignment}
    room_to_weapon = {room: weapon for person, room, weapon in assignment}

    # Clue 1: The man in the kitchen was not found with the rope, knife, or bag.
    if person_to_room.get("George", "") == "Kitchen" or \
       person_to_room.get("John", "") == "Kitchen" or \
       person_to_room.get("Robert", "") == "Kitchen":
        kitchen_weapon = room_to_weapon.get("Kitchen", "")
        if kitchen_weapon in ["Rope", "Knife", "Bag", "Firearm"]:
            return False
    
    # Clue 2: Barbara was either in the study or the bathroom; Yolanda was in the other.
    if person_to_room.get("Barbara", "") not in ["Study", "Bathroom", ""]:
        return False
    if person_to_room.get("Yolanda", "") not in ["Study", "Bathroom", ""]:
        return False
    
    # Clue 3: The person with the bag, who was not Barbara nor George, was not in the bathroom nor the dining room.
    for person, room, weapon in assignment:
        if weapon == "Bag" and (person == "Barbara" or person == "George"):
            return False
        if weapon == "Bag" and room in ["Bathroom", "Dining Room"]:
            return False

    # Clue 4: The woman with the rope was found in the study.
    for person, room, weapon in assignment:
        if weapon == "Rope" and room != "Study":
            return False

    # Clue 5: The weapon in the living room was found with either John or George.
    living_room_weapon = room_to_weapon.get("Living Room", "")
    if living_room_weapon and person_to_room.get("John", "") != "Living Room" and \
       person_to_room.get("George", "") != "Living Room":
        return False
    
    # Clue 6: The knife was not in the dining room.
    if room_to_weapon.get("Dining Room", "") == "Knife":
        return False

    # Clue 7: Yolanda was not with the weapon found in the study nor the pantry.
    study_weapon = room_to_weapon.get("Study", "")
    pantry_weapon = room_to_weapon.get("Pantry", "")
    if person_to_room.get("Yolanda", "") in ["Study", "Pantry"]:
        return False

    # Clue 8: The firearm was in the room with George.
    if person_to_weapon.get("George", "") not in ["Firearm", ""]:
        return False
    
    # Final clue: Mr. Boddy was gassed in the pantry, and the suspect found in that room is the murderer.
    if room_to_weapon.get("Pantry", "") not in ["Gas", ""]:
        return False
    
    return True

# Backtracking algorithm
def backtrack(assignment):
    """
    Perform backtracking to solve the puzzle.
    """
    if len(assignment) == 6:
        return assignment
    
    # Try all rooms and weapons
    for person in people:
        if person not in [p for p, r, w in assignment]:
            for room in rooms:
                if room not in [r for p, r, w in assignment]:
                    for weapon in weapons:
                        if weapon not in [w for p, r, w in assignment]:
                            new_assignment = assignment + [(person, room, weapon)]
                            if is_valid(new_assignment):
                                result = backtrack(new_assignment)
                                if result:
                                    return result
    
    return None

# Find the solution
solution = backtrack(assignments)

# Print the solution
if solution:
    print("Solution found:")
    for person, room, weapon in solution:
        print(f"{person} was in the {room} with the {weapon}")
    print('Num Checks', num_checks)
else:
    print("No solution found.")

Solution found:
George was in the Dining Room with the Firearm
John was in the Living Room with the Bag
Robert was in the Kitchen with the Poison
Barbara was in the Study with the Rope
Christine was in the Pantry with the Gas
Yolanda was in the Bathroom with the Knife
Num Checks 46858


And there you have it! All that's left is to track down this evil Dr. Bui and bring him to justice! You down your drink and start the long trek back to your office, with any luck you'll be home before the bar across the street from your apartment closes.